In [ ]:
import sys
sys.path.append('../')
%load_ext autotime

from tqdm import tqdm
import numpy as np
from fluo.speckle1d import Fluorescence1D

In [ ]:
num_samples = int(1e4)
num_pix = 101
Phi_dim = ((num_pix//2 + 1)//2 + 1)

# Randomizing Phases

In [ ]:
print(f"\nGenerating {num_samples} samples...")
Phi_samples = np.zeros((num_samples, Phi_dim, Phi_dim))
phase_samples = np.zeros((num_samples, num_pix))
for i in tqdm(range(num_samples)):
    # Generate random phase
    phase = np.random.uniform(-np.pi, np.pi, num_pix // 2)
    phase = np.concatenate((-phase, np.zeros(1), np.flip(phase)))
    phase_samples[i, :] = phase
    
    # Compute Phi matrix
    Phi = Fluorescence1D.compute_Phi_from_phase(phase[num_pix // 2:])    
    Phi_samples[i, :, :] = Phi

In [ ]:
print("Mean", np.mean(Phi_samples))
print("StdDev", np.std(Phi_samples))
print("RMS", np.mean(4*Phi_samples**2))

print("Mean", np.mean(phase_samples))
print("StdDev", np.std(phase_samples))
print("RMS", np.mean(4*phase_samples**2))

This is quite fast, maybe for this round of pre-training I should be generating data on the fly?

# Randomizing Coordinates

In [ ]:
print(f"\nGenerating {num_samples} samples...")
Phi_samples = np.zeros((num_samples, Phi_dim, Phi_dim))
phase_samples = np.zeros((num_samples, num_pix))
fluo = Fluorescence1D(kmax=3, num_pix=num_pix, num_atoms=4)
for i in tqdm(range(num_samples)):
    fluo.num_atoms = np.random.randint(3, 20)
    fluo.randomize_coords()
    phase_samples[i, :] = fluo.coh_phase
    _, Phi_samples[i, :, :] = fluo.cosPhi_from_phase() # returned value here is absPhi


In [ ]:
print("Mean", np.mean(Phi_samples))
print("StdDev", np.std(Phi_samples))
print("RMS", np.mean(4*Phi_samples**2))

print("Mean", np.mean(phase_samples))
print("StdDev", np.std(phase_samples))
print("RMS", np.mean(4*phase_samples**2))

This method is slower, because we are doing the same computation for the random phases above but adding a process that computes particular phases. If we wanted to generate inputs on-the-fly to save memory, we could just compute a bunch of these "special" phases and perform the last (fast) step to compute Phi in the DataLoader.

Additionally, we could probably make randomize_coords a little bit more efficient and check cProfile results. Probably not a lot to gain though.

# Computing $g^{(3)}$

In [ ]:
num_shots = 1000

In [ ]:
fluo = Fluorescence1D(kmax=3, num_pix=num_pix, num_atoms=4)
for i in tqdm(range(num_samples)):
    fluo.num_atoms = np.random.randint(3, 20)
    fluo.randomize_coords()
    g3 = fluo.marginalize_g3(num_shots=num_shots)
    # We only need a small slice of this, can we avoid generating the whole 2*num_pix by 2*num_pix array?

If we are memory limited in how many g3 we can generate (if we have to pass through the 3D/6D array) or store (large datasets going as detector area squared), how efficient is it to generate this final round of fine-tuning data on-the-fly?

$g^{(3)}$ Discretization Errors